# Decision Tree Implementation

## making functions required for decision tree and storing the output in a dictionary

In [141]:
#entropy fuction
def Entropy(y):
    value,count = np.unique(y,return_counts = True)  # to get the unique value and its count
    entropy=0
    for i in range(len(value)):
        a= count[i]/count.sum()
        entropy += (-a)*math.log2(a)
    
    return entropy

# for weighted entrop after splitting as per specific feature
def weighted_entropy(x,feature,y):       
    w_entropy = 0
    value,count= np.unique(x[feature],return_counts=True)  # to get the unique value and its count
    for i in range(len(value)):
        a= Entropy(y[x[feature] == value[i]].dropna())  # getting the value of those y whose index is matches the x[feature]==value[i]
        w_entropy += (count[i]/count.sum())*a
    return w_entropy

# function to find information gain
def info_gain(x,feature,y):
    total_entropy = Entropy(y)
    Information_Gain = total_entropy - weighted_entropy(x,feature,y)
    return Information_Gain

# function to find split information
def split_ratio(x,feature):
    value,count = np.unique(x[feature],return_counts = True)  # to get the unique value and its count
    entropy=0
    for i in range(len(value)):
        a= count[i]/count.sum()
        entropy += (-a)*math.log2(a)
    return entropy

# function to find gain ratio
def gain_ratio(info_gain,x,feature):
    ratio =  info_gain/split_ratio(x,feature)
    return ratio

# function for decision tree which will print the required ouput and also return a tree like dictionary
def DecisionTree(x,y,features,target_name,main = None,level=0):    
 
    if len(np.unique(y)) <= 1:   # checking if leaf node is reached
        value,count = np.unique(y,return_counts = True) # to get the unique value and its count
        print('Level ',level)
        print('count of',target_name[int(value[0])],'=',count[0]) # printing the target name and its count
        print('Current Entropy is =',Entropy(y)) # pritnting current entropy
        print('Reached Leaf Node ')
        print()
        
        v,c = np.unique(y,return_counts = True)
        return target_name[int(v[0])],c[0] # returning the target name and its count so that is can be store inside tree dictionary
        
    elif len(features) == 0 :  # checking if features for further split are available or not
        return main # returning the unique value in target as there is no further splitting is possible
    
    else :

        main = []
        v,c = np.unique(y,return_counts = True)
        for i in range(len(v)):
            main.append([target_name[int(v[i])],c[i]]) # getting all the unique data of target name
        
        # checking over all the feature to get the best feature having the max information gain
        values = []
        for f in features :                        
            v = info_gain(x,f,y)     # making the list of information gain for all features
            values.append(v)
        
        
        best_feature_index = np.argmax(values)        
        best_feature = features[best_feature_index] # getting the best feature amongst all
        
        ratio_gain = gain_ratio(max(values),x,best_feature) # getting the gain ratio for the best feature upon which splitting has to be done
        
        tree = {best_feature:{}} # dictionary to store decision tree
        
        tot_entropy = Entropy(y)   # gettnig total entropy for current level
       
                
        value,count = np.unique(y,return_counts = True) # to get the unique value and its count
        print('Level ',level)  
        # printing the target name and its count at given level
        for i in range(len(value)):
            if value[i]==0 :
                print('count of',target_name[int(value[i])]  ,'=',count[i])
            elif value[i]==1 :
                print('count of',target_name[int(value[i])]  ,'=',count[i])
            elif value[i]==2 :
                print('count of',target_name[int(value[i])]  ,'=',count[i])
      
        print('Current entropy is   = ',tot_entropy)  # printing total entropy of current level
        
        print('Splitting on feature ',best_feature,' with gain ratio ', ratio_gain) # printing the gain raion with the splitting feature
        
        print()
        
        # removing the feature after splitting has been done
        new_features = features         
        features=[]
        for i in new_features :         
            if i != best_feature :                  
                features.append(i)
        level += 1       
        new_features=None   # removed the feature which has splitted                  
        
        # for all different value in that particular splitting feature again 
        for i in np.unique(x[best_feature]):  
            x_new = (x.where(x[best_feature] == i)).dropna()
            y_new = (y.where(x[best_feature] == i)).dropna()
            new_tree = DecisionTree(x_new,y_new,features,target_name,main,level) # making tree for other feature
            tree[best_feature][i] = new_tree #updating the tree dictionary
        
        return tree

## importing data and classifying the data as we have continuous data

In [58]:
# importing required libraries
import numpy as np
import pandas as pd
import math
from sklearn import datasets
pd.options.mode.chained_assignment = None  # to remove the error while coping data in dataframe

data = datasets.load_iris() #loading IRIS datasets

x = pd.DataFrame(data.data) #taking all the input value in x and then giving them title
x.columns = ["sl", "sw", 'pl', 'pw']
y = pd.DataFrame(data.target) # taking the output in Y 
y.columns = ['target']
target_name = data.target_names # all the target name
feature_name=x.columns #all the feature name

# a function which will classify the continuous data present in input dataframe
# here classification is done by 4 parts

def classify(df, col):
    second = df[col].mean() #second boundary
    minimum = df[col].min()
    first = (minimum + second)/2 #first boundary
    maximum = df[col].max()
    third = (maximum + second)/2 # third Boundary

# checking each value in column of dataset and using boundaries changing the value and hence making a classified column
 
    for i in range(len(df[col])): 
        if df[col][i]<first:
            df[col][i] = 'a'
        elif df[col][i]<second:
            df[col][i] = 'b'
        elif df[col][i]<third:
            df[col][i] = 'c'
        else:
            df[col][i] = 'd'
            
    return

# using classify function for classification
classify(x, 'sl')
classify(x, 'sw')
classify(x, 'pl')
classify(x, 'pw')

## output for whole dataset

In [142]:
tree=DecisionTree(x,y,feature_name,target_name)

Level  0
count of setosa = 50
count of versicolor = 50
count of virginica = 50
Current entropy is   =  1.584962500721156
Splitting on feature  pw  with gain ratio  0.6996382036222091

Level  1
count of setosa = 50
Current Entropy is = 0.0
Reached Leaf Node 

Level  1
count of versicolor = 10
Current Entropy is = 0.0
Reached Leaf Node 

Level  1
count of versicolor = 40
count of virginica = 16
Current entropy is   =  0.863120568566631
Splitting on feature  pl  with gain ratio  0.4334099495621067

Level  2
count of versicolor = 1
Current Entropy is = 0.0
Reached Leaf Node 

Level  2
count of versicolor = 39
count of virginica = 8
Current entropy is   =  0.6581912658132185
Splitting on feature  sl  with gain ratio  0.12674503775809332

Level  3
count of virginica = 1
Current Entropy is = 0.0
Reached Leaf Node 

Level  3
count of versicolor = 14
Current Entropy is = 0.0
Reached Leaf Node 

Level  3
count of versicolor = 23
count of virginica = 7
Current entropy is   =  0.783776947484701
Sp

In [147]:
tree

{'pw': {'a': ('setosa', 50),
  'b': ('versicolor', 10),
  'c': {'pl': {'b': ('versicolor', 1),
    'c': {'sl': {'a': ('virginica', 1),
      'b': ('versicolor', 14),
      'c': {'sw': {'a': [['versicolor', 23], ['virginica', 7]],
        'b': [['versicolor', 23], ['virginica', 7]],
        'c': ('versicolor', 6)}},
      'd': ('versicolor', 2)}},
    'd': ('virginica', 8)}},
  'd': ('virginica', 34)}}